# core

> This is the core module, I haven't decided if I need any other modules yet

List of things we'd like to log:

* Code version
* Uncommitted changes
* Tree directory snapshot
* All shell output logs
* Time and date
* Docker container info
* All code using wandb to grab code
* Any config files mentioned in argv
* nvidia-smi (this is in some experiments anyway)

This is mostly a wandb wrapper, with the goal being to copy the wandb directory to some location (such as Onedrive) where it can be stored indefinitely, along with a UUID identifier. At the same time it writes the output log and short job info to a shared git repo. Then we can commit these logs and use the git repo as a continuous log of all experiments run, with the UUIDs available so it would be possible to ask for more info from whoever ran the experiment and they may have it saved.

It might become a huge storage hog but I'm willing to take that risk.

## Example Usage


## Structure

Before running anything you run `profane setup` to specify where to save:

* Complete logs (some directory with enough storage space)
* Minimal shared logs (should be a shared git repo)

`profane.core.init` is the entrypoint, it's supposed to act like `wandb.init`.

When initialized:

* It uses the run ID generated by wandb as the run name
* Creates output directory according to the name in the users subdirectory in the shared repository
* Creates mirrored output directory in the local complete log directory

When the run finishes:

* Extracts the `output.log` and saves it to the shared log directory from before, along with the command run and the metadata
* Copies the entire `wandb` run directory to complete log directory

To do this it has to add a teardown hook that will run on the run finishing.
Testing that this definitely works in a standalone script will be annoying.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): pass

In [ ]:
#| export
import wandb
from typing import Callable, NamedTuple
from enum import IntEnum

class ProfaneTeardownStage(IntEnum):
    EARLY = 1
    LATE = 2

class ProfaneTeardownHook(NamedTuple):
    call: Callable[[], None]
    stage: ProfaneTeardownStage

def init(**kwargs):
    """
    A wrapper for `wandb.init`.
    """
    kwargs['mode'] = 'offline'
    run = wandb.init(**kwargs) # always offline
    run._teardown_hooks.append(ProfaneTeardownHook(lambda: print("hook executed"), ProfaneTeardownStage.LATE))
    return run

In [ ]:
#| export
from wandb.proto import wandb_internal_pb2
from wandb.sdk.internal import datastore


def parse_output_log(data_path):
    """
    Parse wandb data from a given path.
    Returns the terminal log typically saved as `output.log`,
    which isn't created unless you're running in online mode.
    But, the data still exists in the `.wandb` file.
    """
    # https://github.com/wandb/wandb/issues/1768#issuecomment-976786476 
    ds = datastore.DataStore()
    ds.open_for_scan(data_path)
    terminal_log = []

    data = ds.scan_record()
    while data is not None:
        pb = wandb_internal_pb2.Record()
        pb.ParseFromString(data[1])  
        record_type = pb.WhichOneof("record_type")
        if record_type == "output_raw":
            terminal_log.append(pb.output_raw.line)
            #print(pb.output_raw)
        data = ds.scan_record()
    return "".join(terminal_log)

In [ ]:
# to test this I need to make a temporary directory and run a fake experiment
# then I can check that whatever gets printed is exactly what is stored in the
# wandb file
from tempfile import TemporaryDirectory
from pathlib import Path
import os
os.environ['WANDB_SILENT'] = 'true'

# import subprocess

printed = []
def _print(*args, **kwargs):
    global printed
    printed += [*args, "\n"]
    return print(*args, **kwargs)

def test_experiment():
    global printed
    with TemporaryDirectory() as tmpdirname:
        print('created temporary directory', tmpdirname) # lol this is from the docs
        tmpdirname = Path(tmpdirname)
        # wandb by default saves to a directory in the current working directory so
        # we need to pass a path to the directory we just created
        target_dir = tmpdirname / 'wandb'
        print(target_dir)
        run = init(dir=tmpdirname)
        wandb.log({'test': 1})
        _print("something to be logged")
        for i in range(10):
            _print(f"logging {i}")
        run.finish()
        # list the files in the directory
        for path_object in tmpdirname.rglob('*'):
            if path_object.is_file():
                if path_object.suffix == '.wandb':
                    print("found wandb file ", path_object)
                    #subprocess.run(['wandb', 'sync', '--view', '--verbose', str(path_object)])
                    _printed = parse_output_log(path_object)
        assert "".join(printed) == "".join(_printed)

test_experiment()

created temporary directory /var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmp4ot3x1ij
/var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmp4ot3x1ij/wandb
something to be logged
logging 0
logging 1
logging 2
logging 3
logging 4
logging 5
logging 6
logging 7
logging 8
logging 9
hook executed
found wandb file  /var/folders/ln/1018n5357kjc745b28hf7jzm0000gn/T/tmp4ot3x1ij/wandb/offline-run-20230727_165109-li21mx6o/run-li21mx6o.wandb


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()